In [1]:
import json
import os
from glob import glob

# Objects

- Chỉ lấy những objects có điểm >0.4

In [ ]:
os.makedirs('./objects_filtered')

In [13]:
objects_paths = glob('./objects-b2/objects\\*\\*.json')

In [12]:
path = './objects/L01_V001/0001.json'

path.split('/')[-2] + '/' + path.split('/')[-1]

'L01_V001/0001.json'

In [14]:
for obj_path in objects_paths:
    with open(obj_path, 'r') as r_stream:
        keyframe_filename = obj_path.split('\\')[-2] + '/' + obj_path.split('\\')[-1]
        obj = json.load(r_stream)
        obj_filtered = {
            "Objects": [],
            "Scores": []
        }
        idx = 0
        for score in obj["detection_scores"]:
            if float(score) < 0.7:
                break
            else:
                obj_filtered["Scores"].append(score)
                obj_filtered["Objects"].append(obj["detection_class_entities"][idx])
                idx += 1
        os.makedirs('./objects_filtered/' + keyframe_filename.split('/')[0], exist_ok=True)
        with open('./objects_filtered/' + keyframe_filename, 'w') as w_stream:
            json.dump(obj_filtered, w_stream)

In [ ]:
os.makedirs('./objects_filtered')
objects_paths = glob('./objects\\*\\*.json')

for obj_path in objects_paths:
    with open(obj_path, 'r') as r_stream:
        keyframe_filename = obj_path.split('\\')[-2] + '/' + obj_path.split('\\')[-1]
        obj = json.load(r_stream)
        obj_filtered = {
            "Objects": [],
            "Scores": []
        }
        idx = 0
        for score in obj["detection_scores"]:
            if float(score) < 0.4:
                break
            else:
                obj_filtered["Scores"].append(score)
                obj_filtered["Objects"].append(obj["detection_class_entities"][idx])
                idx += 1
        os.makedirs('./objects_filtered/' + keyframe_filename.split('/')[0], exist_ok=True)
        with open('./objects_filtered/' + keyframe_filename, 'w') as w_stream:
            json.dump(obj_filtered, w_stream)

# Concept detection

In [5]:
import azure.storage.blob as asb
import os
connectionString = ""

#Kết nối với Azure Storage
service = asb.BlobServiceClient.from_connection_string(conn_str=connectionString)

In [6]:
folder_name = '20/' #Chỉnh cái này để chuyển folder

In [7]:
keyf_container = service.get_container_client("keyframes")
# cur_blobs = keyf_container.walk_blobs('01/keyframes/', delimiter='/')
cur_blobs = keyf_container.list_blob_names(name_starts_with=folder_name)

In [8]:
# Tải tầm 15 phút
for blob in cur_blobs:
    #Tải keyframes
    blob_data = keyf_container.download_blob(blob)
    #Tạo folder chứa keyframes
    path = 'keyframes/' + blob[:blob.rfind('/')]
    if not os.path.exists(path):
        os.makedirs(path)
    #Tải file vào folder
    with open('keyframes/' + blob, 'wb') as p:
        blob_data.readinto(p)

Unable to stream download: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))


In [9]:
import json

with open('./concepts_config.json', 'r') as r_stream:
    id2label = json.load(r_stream)

In [10]:
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import torch
from PIL import Image
from glob import glob

keyframes_path = glob("./keyframes\\*\\*\\*\\*.jpg")

preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b2")
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b2")

for path in keyframes_path:
    image = Image.open(path)
    vid_name = path.split('\\')[3]
    frameid = path.split('\\')[-1][:4]
    dest_path = "./concepts/" + vid_name + '/' 

    inputs = preprocessor(image, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    # model predicts one of the 1000 ImageNet classes
    predicted_label = logits.argmax(-1).item()
    # print(model.config.id2label[predicted_label])
    # concept_label.append(model.config.id2label[predicted_label])
    concept_label = id2label[str(predicted_label)]
    
    os.makedirs(dest_path, exist_ok=True)
    with open(dest_path + frameid + '.json', 'w') as w_stream:
        json.dump(concept_label, w_stream)

# Fix naming

In [16]:
folders = ['L19','L20']
A = ['V001','V002']
concepts_path = glob('./concepts/*\\*.json')

for path in concepts_path:
    if (path[11] + path[12] + path[13] in folders) and (path[15]+path[16]+path[17]+path[18] in A):
        if(path[11:14] == 'L19') and (path[15:19] == 'V001'):
            continue 
        newname = path[:19] + '/' + path[20] + path[21] + path[22] + '.json'
        os.rename(path, newname)


# *Playgrounds*

In [11]:
objects_paths = glob('./objects_filtered/*')
clip_paths = glob('./clip-features-vit-b32/*.npy')

obj_vids = []
for path in objects_paths:
    obj_vids.append(path.split('\\')[-1])
    
for path in clip_paths:
    vid = path.split('\\')[-1][:8]
    if vid not in obj_vids:
        print(vid)

L11_V007
L12_V011
L12_V028
L13_V002
L14_V015
L14_V026
L15_V007
L15_V019
L16_V030
L19_V060
L19_V098
L20_V059
L20_V083


In [ ]:
import pickle

with open('unsplash-25k-photos-embeddings.pkl', 'rb') as fIn:
        img_names, img_emb = pickle.load(fIn) 

In [ ]:
img_emb[0]

In [ ]:
import torch
from datasets import load_dataset
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
from imageio import imopen

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]
print(image)

preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b2")
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b2")

inputs = preprocessor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])
